In [1]:
import pandas as pd
import xlsxwriter
import numpy as np

import nlpaug.augmenter.word.context_word_embs as aug

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


from tqdm.auto import tqdm



C:\Users\MOHSEN\anaconda3\envs\mytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#read the data into a pandas dataframe
df = pd.read_csv("bert_data_withoutClassWord.csv")
print(df.shape)
df.head(5)

(7115, 3)


,Unnamed: 0,text,class
0,0,this article is about the herbivorous mammals....,Antelope
1,1,"one new world species, the pronghorn of north ...",Antelope
2,2,"the english word ""animal"" first appeared in 14...",Antelope
3,3,"the word talopus and calopus, from latin, came...",Antelope
4,4,animal are not a cladistic or taxonomically de...,Antelope


In [3]:
#Add the new column which gives a unique number to each of these labels 

j = 0
for i in df['class'].unique():
    df.loc[df['class'] == i, ['class_num']] = j
    j += 1

#checking the results 
df.head(50000)

,Unnamed: 0,text,class,class_num
0,0,this article is about the herbivorous mammals....,Antelope,0.0
1,1,"one new world species, the pronghorn of north ...",Antelope,0.0
2,2,"the english word ""animal"" first appeared in 14...",Antelope,0.0
3,3,"the word talopus and calopus, from latin, came...",Antelope,0.0
4,4,animal are not a cladistic or taxonomically de...,Antelope,0.0
...,...,...,...,...
7110,7110,technology to use sponges as mouth protection ...,dolphin,49.0
7111,7111,"pesticides, heavy metals, plastics, and other ...",dolphin,49.0
7112,7112,"hundreds of orcas, animals and other members o...",dolphin,49.0
7113,7113,captured orcas and animals are confined to tan...,dolphin,49.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.class_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.class_num
)

In [5]:
embbeding_size = 8000
clustering_count = 3
numof_embb = 2
att_size = embbeding_size * clustering_count

In [6]:
vectorizer = TfidfVectorizer(max_features = embbeding_size).fit(X_train)
        
X_train_vectorized = vectorizer.transform(X_train)
    
clf = RandomForestClassifier()
clf.fit(X_train_vectorized, y_train)
    
predictions = clf.predict(vectorizer.transform(X_test))

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.29      0.39      0.34        38
         1.0       0.40      0.26      0.32        23
         2.0       0.70      0.57      0.63        28
         3.0       0.45      0.54      0.49        37
         4.0       0.11      0.04      0.06        23
         5.0       0.27      0.46      0.34        37
         6.0       0.39      0.61      0.47        44
         7.0       0.36      0.61      0.45        31
         8.0       0.45      0.43      0.44        23
         9.0       0.35      0.28      0.31        25
        10.0       0.45      0.24      0.31        21
        11.0       0.33      0.20      0.25        20
        12.0       0.39      0.36      0.38        36
        13.0       0.68      0.66      0.67        29
        14.0       0.56      0.34      0.43        29
        15.0       0.52      0.56      0.54        27
        16.0       0.33      0.08      0.13        12
        17.0       0.31    

In [7]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

In [8]:
df['class_num'].value_counts()


23.0    224
6.0     219
29.0    204
31.0    196
18.0    193
0.0     190
28.0    185
42.0    184
5.0     184
3.0     182
12.0    180
44.0    176
38.0    165
49.0    165
39.0    164
19.0    162
33.0    156
22.0    155
7.0     154
37.0    154
14.0    145
13.0    143
40.0    143
30.0    142
48.0    141
2.0     140
47.0    139
15.0    136
24.0    131
36.0    130
32.0    128
45.0    127
46.0    124
9.0     123
35.0    122
27.0    118
8.0     117
1.0     117
4.0     116
43.0    112
25.0    110
26.0    107
17.0    107
21.0    107
10.0    106
41.0    101
11.0     98
34.0     73
20.0     61
16.0     59
Name: class_num, dtype: int64

In [9]:
def augmentMyData(df, augmenter, class_number, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['class_num'] == class_number].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['text'].iloc[i])
            #print(augmented_text)
            #print(str(augmented_text)[2:-2])
            augmented_texts.append(str(augmented_text)[2:-2])
    
    data = {
        'class_num': class_number,
        'text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [10]:
new_df = df[['text', 'class_num']]
new_df.head(5)

,text,class_num
0,this article is about the herbivorous mammals....,0.0
1,"one new world species, the pronghorn of north ...",0.0
2,"the english word ""animal"" first appeared in 14...",0.0
3,"the word talopus and calopus, from latin, came...",0.0
4,animal are not a cladistic or taxonomically de...,0.0


In [11]:
class_count = new_df['class_num'].unique().shape[0]
class_with_maxCount = new_df['class_num'].value_counts().max()

for i in range(class_count):
    
    samples = class_with_maxCount - new_df['class_num'].value_counts()[i]
    
    print(f'augmenting {samples} for class of {i}')
    new_df = augmentMyData(new_df, augmenter, class_number=i, samples=samples)

augmenting 34 for class of 0


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:27<00:00,  1.23it/s]
C:\Users\MOHSEN\AppData\Local\Temp\ipykernel_71224\2429172311.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(aug_df).reset_index(drop=True))


augmenting 107 for class of 1


100%|████████████████████████████████████████████████████████████████████████████████| 107/107 [01:28<00:00,  1.21it/s]
C:\Users\MOHSEN\AppData\Local\Temp\ipykernel_71224\2429172311.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(aug_df).reset_index(drop=True))


augmenting 84 for class of 2


...

100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [01:16<00:00,  1.11it/s]
C:\Users\MOHSEN\AppData\Local\Temp\ipykernel_71224\2429172311.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(aug_df).reset_index(drop=True))


augmenting 83 for class of 48


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [01:16<00:00,  1.08it/s]
C:\Users\MOHSEN\AppData\Local\Temp\ipykernel_71224\2429172311.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(aug_df).reset_index(drop=True))


augmenting 59 for class of 49


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:52<00:00,  1.12it/s]
C:\Users\MOHSEN\AppData\Local\Temp\ipykernel_71224\2429172311.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = shuffle(df.append(aug_df).reset_index(drop=True))


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    new_df.text, 
    new_df.class_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=new_df.class_num
)


embbeding_size = 8000
clustering_count = 3
numof_embb = 2
att_size = embbeding_size * clustering_count



vectorizer = TfidfVectorizer(max_features = embbeding_size)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_train_vectorized = X_train_vectorized.toarray()
            
clf = RandomForestClassifier()
clf.fit(X_train_vectorized, y_train)
    
predictions = clf.predict(vectorizer.transform(X_test))

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.64      0.62      0.63        45
         1.0       0.75      0.84      0.79        45
         2.0       0.88      0.82      0.85        45
         3.0       0.55      0.66      0.60        44
         4.0       0.97      0.62      0.76        45
         5.0       0.55      0.73      0.63        45
         6.0       0.48      0.48      0.48        44
         7.0       0.56      0.86      0.68        44
         8.0       0.78      0.84      0.81        45
         9.0       0.76      0.77      0.76        44
        10.0       0.95      0.84      0.89        45
        11.0       0.89      0.91      0.90        45
        12.0       0.79      0.51      0.62        45
        13.0       0.97      0.82      0.89        45
        14.0       0.80      0.62      0.70        45
        15.0       0.69      0.82      0.75        45
        16.0       0.80      0.98      0.88        45
        17.0       0.85    

In [19]:
type(y_train)
y_train
t

,text,class_num
1888,animales tend typically to have an average lit...,21.0
2347,tests recorded using immature mice ( notably t...,43.0
3853,animal were one of the first and most successf...,22.0
10084,suppose a typical animal has a large head shap...,41.0
7176,according mostly to those journalists who also...,45.0
...,...,...
215,animals communicate through more than just cli...,49.0
1140,there are many names for gray wolves besides t...,31.0
9233,animals are in a group of breeds classed as ‘c...,7.0
1601,these pups are born hairless and blind in a ne...,11.0


In [ ]:
# split the dataset before augmenting to avoid augmented data in valid set
X_train, _, y_train, _ = train_test_split(aug_df['text'], aug_df['class_num'].values, test_size=0.2)
_, X_test, _, y_test = train_test_split(df['text'], df['class_num'].values, test_size=0.5)
